# Working with Hierarchies in Atoti

In this notebook, we'll show you what you need to know to start working with hierarchies when using the Atoti Python API.

Before we begin, let's start off with a refresher from our Atoti Python API [Tutorial](https://docs.atoti.io/latest/getting_started/tutorial/tutorial.html#Multidimensional-concepts):

> A **cube** is a multidimensional view of some data, making it easy to explore, aggregate, filter and compare. It’s called a cube because each attribute of the data can be represented as a dimension of the cube. The axes of the cube are called **hierarchies**. The purpose of multidimensional analysis is to visualize some numeric indicators at specific coordinates of the cube. These indicators are called **measures**.

In other words, the purpose of an OLAP cube is to provide multidimensional analysis capabilities, which gives us the flexibility to slice-and-dice our data however we see fit. With this being said, let's talk about hierarchies!

💡 **Note:** Many of the initial operations in this notebook revolve around [Table Creation and Data Loads](../table-creation-and-data-loads/main.ipynb). Feel free to check out the notebook if you need some reminders on creating, loading, and joining Atoti tables.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=hierarchies" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Discover-Atoti-now.png" alt="Try Atoti"></a></div>

## Import libraries

First things first, let's import our libraries.

In [1]:
import atoti as tt
import pandas

## Instantiate Atoti server and link UI

Next, we'll spin up the Atoti server and link the session to our web-based Atoti UI.

💡 **Note**: https://docs.atoti.io/latest/api/atoti.Session.html#atoti.Session

In [2]:
session = tt.Session(
    user_content_storage="./content",
    port=9092,
    java_options=["-Xms1G", "-Xmx10G"],
)
session.link

http://localhost:9092

_Note_: This is the session's local URL: it may not be reachable if Atoti is running on another machine.

## Load CSVs from S3 into Atoti tables

From here on, we'll load the following CSVs from S3 into Atoti tables.

* [Orders](#Orders)
* [Products](#Products)
* [Shippers](#Shippers)
* [Customers](#Customers)
* [Employees](#Employees)

💡 **Note**: Sometimes we want inherently numerical columns to be used as hierarchies instead of measures (e.g. an Order ID should be used as a hierarchy rather than a measure, and does not need to be aggregated). For this reason, we can explicitly set specific Atoti table columns as desired data types (in this case strings) when loading data into Atoti tables.

See https://docs.atoti.io/latest/api/atoti.Session.read_csv.html#atoti.Session.read_csv for more info.

### Orders

In [3]:
orders = session.read_csv(
    "s3://data.atoti.io/notebooks/hierarchies/data/Orders.csv",
    types={
        "OrderId": tt.STRING,
        "OrderDate": tt.LOCAL_DATE,
        "ProductId": tt.STRING,
        "EmployeeId": tt.STRING,
        "CustomerId": tt.STRING,
    },
    date_patterns={"OrderDate": "dd/M/yyyy"},
    keys=["OrderId"],
)
orders.head()

,OrderDate,QuantitySold,SellingPricePerUnit,ShipperName,ProductId,EmployeeId,CustomerId
OrderId,,,,,,,
9,2024-02-18,339,19.39,The Transporter,1423,18336,c5755
19,2024-02-18,971,11.1,Freight United,708,18422,c1891
29,2024-02-18,716,12.56,Rapid Post,1202,18465,c3235
39,2024-02-18,120,284.01,Speedy Express,1410,18508,c3179
49,2024-02-18,471,51.67,United Package,1111,26584,c2003


### Products

In [4]:
products = session.read_csv(
    "s3://data.atoti.io/notebooks/hierarchies/data/Products.csv",
    types={
        "ProductId": tt.STRING,
        "ProductName": tt.STRING,
        "ProductCategory": tt.STRING,
        "Supplier": tt.STRING,
        "PurchasingPricePerUnit": tt.DOUBLE,
    },
    keys=["ProductId"],
)
products.head()

,ProductName,ProductCategory,Supplier,PurchasingPricePerUnit
ProductId,,,,
643,Ravioli Angelo,Grains/Cereals,Pasta Buttini s.r.l.,11.23
773,Chang,Beverages,Exotic Liquids,10.26
903,Gravad lax,Seafood,Svensk Sjöföda AB,18.72
1033,Lakkalikööri,Beverages,Karkki Oy,9.72
1163,Rhönbräu Klosterbier,Beverages,Plutzer Lebensmittelgroßmärkte AG,4.19


### Shippers

In [5]:
shippers = session.read_csv(
    "s3://data.atoti.io/notebooks/hierarchies/data/Shippers.csv",
    types={
        "ShipperName": tt.STRING,
        "Contact": tt.STRING,
    },
)
shippers.head()

,ShipperName,Contact
0,Freight United,Geoff Matthews
1,Rapid Post,Payal Shah
2,United Package,Junqi Huang
3,Speedy Express,Natalia Ramirez
4,The Transporter,Funmi Odunga


### Customers

In [6]:
customers = session.read_csv(
    "s3://data.atoti.io/notebooks/hierarchies/data/Customers.csv",
    types={
        "CustomerId": tt.STRING,
        "CompanyName": tt.STRING,
        "Region": tt.STRING,
        "Country": tt.STRING,
        "Address": tt.STRING,
        "City": tt.STRING,
        "PostCode": tt.STRING,
        "Phone": tt.STRING,
    },
    keys=["CustomerId"],
)
customers.head()

,CompanyName,Address,City,PostalCode,Region,Country,Subregion,Phone,PostCode
CustomerId,,,,,,,,,
c883,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,51100,Europe,France,Western Europe,26.47.15.10,N/A
c939,Toms Spezialitäten,Luisenstr. 48,Münster,44087,Europe,Germany,Western Europe,0251-031259,N/A
c5643,Alfreds Futterkiste,Obere Str. 57,Berlin,12209,Europe,Germany,Western Europe,030-0074321,N/A
c2899,Godos Cocina Típica,"C/ Romero, 33",Sevilla,41101,Europe,Spain,Southern Europe,(95) 555 82 82,N/A
c4019,Seven Seas Imports,90 Wadhurst Rd.,London,OX15 4NB,Europe,United Kingdom,Northern Europe,(171) 555-1717,N/A


### Employees

In [7]:
employees = session.read_csv(
    "s3://data.atoti.io/notebooks/hierarchies/data/Employees.csv",
    types={
        "EmployeeId": tt.STRING,
        "EmployeeName": tt.STRING,
        "EmployeeCountry": tt.STRING,
        "EmployeeCity": tt.STRING,
    },
    keys=["EmployeeId"],
)
employees.head()

,EmployeeName,EmployeeZone,EmployeeCountry,EmployeeCity
EmployeeId,,,,
18594,Laura Callahan,AMERICAS,USA,Seattle
18637,Andrew Fuller,AMERICAS,USA,Tacoma
12344,Paul Henry,EMEA,FR,Paris
18680,Robert King,EMEA,UK,London
12695,Louis Philippe,EMEA,FR,Lyon


## Join tables and create cube

Now that our data is loaded into Atoti tables and in the format that we want, we can join our tables together on their common keys and create our Atoti cube.

In [8]:
# Join tables
orders.join(products, orders["ProductId"] == products["ProductId"])
orders.join(employees, orders["EmployeeId"] == employees["EmployeeId"])
orders.join(customers, orders["CustomerId"] == customers["CustomerId"])
orders.join(shippers, orders["ShipperName"] == shippers["ShipperName"])

In [9]:
# Create Cube from Atoti Table object
cube = session.create_cube(orders)

## Analyze the Atoti cube

Now that the data has been loaded into the cube, we'll want to examine the data model of the cube to confirm that the columns and data types match up to what we expect. In addition, we can view the hierarchies, levels, and measures for our cube by assigning them to variables.

In [10]:
# View the schema of the data model
session.tables.schema

```mermaid
erDiagram
  "Orders" {
    _ String PK "OrderId"
    _ LocalDate "OrderDate"
    nullable int "QuantitySold"
    nullable double "SellingPricePerUnit"
    _ String "ShipperName"
    _ String "ProductId"
    _ String "EmployeeId"
    _ String "CustomerId"
  }
  "Products" {
    _ String PK "ProductId"
    _ String "ProductName"
    _ String "ProductCategory"
    _ String "Supplier"
    nullable double "PurchasingPricePerUnit"
  }
  "Customers" {
    _ String PK "CustomerId"
    _ String "CompanyName"
    _ String "Address"
    _ String "City"
    _ String "PostalCode"
    _ String "Region"
    _ String "Country"
    _ String "Subregion"
    _ String "Phone"
    _ String "PostCode"
  }
  "Shippers" {
    _ String "ShipperName"
    _ String "Contact"
  }
  "Employees" {
    _ String PK "EmployeeId"
    _ String "EmployeeName"
    _ String "EmployeeZone"
    _ String "EmployeeCountry"
    _ String "EmployeeCity"
  }
  "Orders" }o--o| "Customers" : "`CustomerId` == `CustomerId`"
  "Orders" }o--o| "Employees" : "`EmployeeId` == `EmployeeId`"
  "Orders" }o--o| "Products" : "`ProductId` == `ProductId`"
  "Orders" }o--o| "Shippers" : "`ShipperName` == `ShipperName`"
```


In [11]:
# Set variables for hierarchies, levels, and measures
h, l, m = cube.hierarchies, cube.levels, cube.measures

## Viewing hierarchies

Because this walk through will mainly be focusing on hierarchies, let's view the hierarchies for the data model by printing the list of hierarchies based on the variable that we assigned.

We can see that each hierarchy is categorized under dimensions, which derive from the Atoti tables that we loaded, and each hierarchy has its own set of levels (starting with a 0-based index).

💡 **Note**: Hierarchies can have [multiple levels](https://docs.atoti.io/latest/getting_started/tutorial/tutorial.html#Multilevel-hierarchies), which we will show [later in this notebook](#Using-Multi-Level-Hierarchies).

In [12]:
h

{('Products', 'Supplier'): <atoti.hierarchy.Hierarchy object at 0x133ea7290>, ('Orders', 'OrderDate'): <atoti.hierarchy.Hierarchy object at 0x133ea6350>, ('Orders', 'ProductId'): <atoti.hierarchy.Hierarchy object at 0x133ea6310>, ('Products', 'ProductName'): <atoti.hierarchy.Hierarchy object at 0x133ea6250>, ('Customers', 'Phone'): <atoti.hierarchy.Hierarchy object at 0x133ea62d0>, ('Customers', 'Address'): <atoti.hierarchy.Hierarchy object at 0x133ea5fd0>, ('Customers', 'Region'): <atoti.hierarchy.Hierarchy object at 0x133e62790>, ('Shippers', 'Contact'): <atoti.hierarchy.Hierarchy object at 0x133e61cd0>, ('Orders', 'EmployeeId'): <atoti.hierarchy.Hierarchy object at 0x133e61010>, ('Customers', 'PostCode'): <atoti.hierarchy.Hierarchy object at 0x133e63c50>, ('Employees', 'EmployeeCountry'): <atoti.hierarchy.Hierarchy object at 0x133e63590>, ('Products', 'ProductCategory'): <atoti.hierarchy.Hierarchy object at 0x133e637d0>, ('Customers', 'PostalCode'): <atoti.hierarchy.Hierarchy object at 0x133e63c10>, ('Customers', 'Subregion'): <atoti.hierarchy.Hierarchy object at 0x133e62e10>, ('Orders', 'CustomerId'): <atoti.hierarchy.Hierarchy object at 0x133e63f50>, ('Employees', 'EmployeeCity'): <atoti.hierarchy.Hierarchy object at 0x133e628d0>, ('Customers', 'Country'): <atoti.hierarchy.Hierarchy object at 0x133e63ed0>, ('Orders', 'OrderId'): <atoti.hierarchy.Hierarchy object at 0x133e612d0>, ('Customers', 'CompanyName'): <atoti.hierarchy.Hierarchy object at 0x133e636d0>, ('Orders', 'ShipperName'): <atoti.hierarchy.Hierarchy object at 0x133e634d0>, ('Employees', 'EmployeeName'): <atoti.hierarchy.Hierarchy object at 0x133e63a90>, ('Employees', 'EmployeeZone'): <atoti.hierarchy.Hierarchy object at 0x133dae710>, ('Customers', 'City'): <atoti.hierarchy.Hierarchy object at 0x133e94a10>}

## Hierarchy basics

Now that we have our cube up and running and have checked the hierarchies present in the cube, let's use `session.widget` to open up an Atoti JupyterLab extension connected to our cube, and investigate the quantity sold per product.

Here we can use the `ProductId` hierarchy and the `QuantitySold.SUM` measure to perform this operation in Atoti.

💡 **Note**: You may also use `session.link` to perform the same investigation within the Atoti UI; however, for the purposes of this walk through, we've elected to use `session.widget` so that we can show our analysis directly within this Jupyter notebook.

In [13]:
session.widget

Open the notebook in JupyterLab with the Atoti JupyterLab extension enabled to build this widget.

But perhaps we also want to know the particular customers that bought each product, along with the quantity sold per product and customer. With Atoti, this is as simple as adding another hierarchy, `CustomerId`, to our analysis to further slice-and-dice our data. As we can see, the total for the `QuantitySold.SUM` measure stays the same at `212,034`, but we can drill down to see a more granular breakdown.

In [14]:
session.widget

Open the notebook in JupyterLab with the Atoti JupyterLab extension enabled to build this widget.

## Using multi-level hierarchies

As we mentioned previously, our hierarchies in the cube currently have one level each, but sometimes there are parent-child relationships that we want to logically group together in Atoti to exemplify these relationships. For example, there is a parent-child relationship between our `ProductCategory` (parent) and `ProductId` (child) hierarchies, as each individual product falls within the umbrella of a higher-level product category.

We can create a new `Product` multi-level hierarchy that uses the underlying levels for the `ProductCategory` and `ProductId` hierarchies.

In [15]:
h["Product"] = [l["ProductCategory"], l["ProductId"]]

As a best practice, we should also clean up the associated single-level hierarchies to avoid redundancy.

In [16]:
# # We've commented this cell to avoid disrupting the
# # upstream widgets that depend on these hierarchies.
# # Uncomment this cell and execute to delete the hierarchies.
# del h['ProductCategory']
# del h['ProductId']

When we take a look back at our list of hierarchies, either in the Atoti JupyterLab extension or the Atoti UI, we can now see our new multi-level hierarchy `Product` with a drop-down to represent it's underlying (multiple) levels.

![multi-level-hierarchies](https://data.atoti.io/notebooks/hierarchies/images/multi-level-hierarchies.png)


And if we now use the `Product` hierarchy in our analysis, we get a breakdown of our data for `QuantitySold.SUM` by associated product categories and product IDs.

In [17]:
session.widget

Open the notebook in JupyterLab with the Atoti JupyterLab extension enabled to build this widget.

## Multi-level date hierarchies

One of the most common use cases for creating multi-level hierarchies is when dealing with dates. For example, perhaps we want to have a multi-level `OrderDate` hierarchy that shows the breakdown for each corresponding year, quarter, month, day, and so on.

As of right now, our single-level `OrderDate` hierarchy can only drill-down to the fully qualified date as shown. 

In [18]:
session.widget

Open the notebook in JupyterLab with the Atoti JupyterLab extension enabled to build this widget.

But if we create a multi-level `OrderDate` hierarchy, we can then show the breakdown for each corresponding year, quarter, month, and day.

💡 **Note**: https://docs.atoti.io/latest/api/atoti.Cube.create_date_hierarchy.html#atoti.Cube.create_date_hierarchy

In [19]:
# Create a multi-level date hierarchy
cube.create_date_hierarchy(
    "Multi-level Date",
    column=orders["OrderDate"],
    levels={"Year": "yyyy", "Quarter": "QQQ", "Month": "MMM", "Day": "dd"},
)

In [20]:
# # We've commented this cell to avoid disrupting the
# # upstream widgets that depend on these hierarchies.
# # Uncomment this cell and execute to delete the hierarchies.
# del h['OrderDate']

In [21]:
session.widget

Open the notebook in JupyterLab with the Atoti JupyterLab extension enabled to build this widget.

## Using hierarchy properties and functions

Each hierarchy is an object of the [atoti.Hierarchy](https://docs.atoti.io/latest/api/atoti.Hierarchy.html#atoti.Hierarchy) class. This means that we are able to interact with and manipulate hierarchies using their associated class properties and functions. Let's focus on using some of the more basic properties and functions.

### Basic hierarchy properties and functions

#### `Hierarchy.dimension`

We can use the `dimension` property of a Hierarchy object to determine its parent dimension. For example, we can call the `dimension` property of the `Address` hierarchy to determine that its parent dimension is `Customers`. 

In [22]:
h["Address"].dimension

'Customers'

#### `Hierarchy.name`

The `name` property will output the name of the hierarchy like so:

In [23]:
h["Address"].name

'Address'

Though this is a bit redundant when we already know the name of the hierarchy. Therefore, using the `name` property may be more helpful when trying to execute a `for` loop to iterate over all hierarchies in an Atoti data model, where we may want the colloquial hierarchy name rather than the fully qualified one (with both the dimension and hierarchy listed).

In [24]:
for key in h.keys():
    print(key)

('Products', 'ProductName')
('Customers', 'PostCode')
('Employees', 'EmployeeCountry')
('Customers', 'PostalCode')
('Customers', 'Subregion')
('Employees', 'EmployeeCity')
('Customers', 'Country')
('Orders', 'OrderId')
('Orders', 'ShipperName')
('Employees', 'EmployeeZone')
('Products', 'Supplier')
('Orders', 'OrderDate')
('Orders', 'ProductId')
('Customers', 'Phone')
('Customers', 'Address')
('Customers', 'Region')
('Orders', 'Multi-level Date')
('Shippers', 'Contact')
('Orders', 'EmployeeId')
('Products', 'ProductCategory')
('Orders', 'CustomerId')
('Products', 'Product')
('Customers', 'CompanyName')
('Employees', 'EmployeeName')
('Customers', 'City')


In [25]:
for key in h.keys():
    print(h[key].name)

ProductName
PostCode
EmployeeCountry
PostalCode
Subregion
EmployeeCity
Country
OrderId
ShipperName
EmployeeZone
Supplier
OrderDate
ProductId
Phone
Address
Region
Multi-level Date
Contact
EmployeeId
ProductCategory
CustomerId
Product
CompanyName
EmployeeName
City


#### `Hierarchy.levels`



We can use the `levels` property to display the available levels for a particular hierarchy. For example, if we were to print the assigned variable that we used for `cube.levels`, this will show us all the available levels for the entire data model.

In [26]:
l

But let's say we wanted to see just the levels for our previous multi-level hierarchy `Product`. Scanning through the fully qualified list of levels may be a bit cumbersome (especially if we have a large amount of levels!). Thus, we can instead use the `levels` property for the multi-level `Product` hierarchy object to only show the levels for that particular hierarchy.

In [27]:
product_levels = h["Product"].levels

for key in product_levels.keys():
    print(key)

ProductCategory
ProductId


In [28]:
product_levels["ProductCategory"]

In [29]:
product_levels["ProductId"]

#### `Hierarchy.dimension_default`

We can also call the `dimension_default` property of a Hierarchy object to determine if a specified hierarchy is the default hierarchy in its parent dimension.

💡 **Note:** https://docs.atoti.io/latest/api/atoti.Hierarchy.dimension_default.html#atoti.Hierarchy.dimension_default

In [30]:
h["Address"].dimension_default

False

The output of the `dimension_default` property for a Hierarchy should be either `True` or `False`. In this case, the `Address` hierarchy of the `Customers` dimension is **not** the default hierarchy for the `Customers` dimension.

So which hierarchy **is** the default hierarchy for the `Customers` dimension? We can create a simple `for` loop to test this. 

💡 **Note:** The `Orders` and `Customers` tables are joined by the common key: `CustomerId`. However, from an Atoti data model perspective, `CustomerId` is a hierarchy present under the base table or `Orders` dimension, and is no longer present as a (redundant) hierarchy under the `Customers` dimension. Therefore, when checking the `Customers` dimension for its default hierarchy, we must filter out the `CustomerId` out from the search criteria — otherwise we get a missing `KeyError`.

In [31]:
def check_default_hierarchy(hierachies, table):
    """Checks for the default hierarchy for an Atoti Table object."""
    dimension = table.name
    for hierarchy in table.columns:
        if hierarchy != "CustomerId":
            print(
                f"h['{dimension}', '{hierarchy}'].dimension_default == {h[dimension, hierarchy].dimension_default}"
            )


check_default_hierarchy(h, customers)

h['Customers', 'CompanyName'].dimension_default == False
h['Customers', 'Address'].dimension_default == False
h['Customers', 'City'].dimension_default == False
h['Customers', 'PostalCode'].dimension_default == True
h['Customers', 'Region'].dimension_default == False
h['Customers', 'Country'].dimension_default == False
h['Customers', 'Subregion'].dimension_default == False
h['Customers', 'Phone'].dimension_default == False
h['Customers', 'PostCode'].dimension_default == False


We can see that the default hierarchy for the `Customers` dimension is actually `PostalCode`. If we set another hierarchy under the `Customers` dimension as the default hierarchy, it would look like this:

In [32]:
h["Address"].dimension_default = True

Then, if we re-use the `check_default_hierarchy` function that we defined previously, we can see that the default hierarchy for the `Customers` dimension has indeed been updated.

In [33]:
check_default_hierarchy(h, customers)

h['Customers', 'CompanyName'].dimension_default == False
h['Customers', 'Address'].dimension_default == True
h['Customers', 'City'].dimension_default == False
h['Customers', 'PostalCode'].dimension_default == False
h['Customers', 'Region'].dimension_default == False
h['Customers', 'Country'].dimension_default == False
h['Customers', 'Subregion'].dimension_default == False
h['Customers', 'Phone'].dimension_default == False
h['Customers', 'PostCode'].dimension_default == False


#### `Hierarchy.visible`

The `visible` property will either display or hide a hierarchy from the list of available hierarchies when using Atoti in either the Atoti JupyterLab extension or the Atoti UI. For example, if we look at our data model, we can see the `CustomerId` hierarchy present in the list of available hierarchies.

💡 **Note:** The `visible` property will **not** delete a hierarchy from an Atoti data model, it merely hides it as an available hierarchy in any of our UI interfaces. Programmatically, a hidden hierarchy can still be used.

<img src="https://data.atoti.io/notebooks/hierarchies/images/hierarchies-visible-true.png" alt="hierarchies-visible-true" width="225"/>

If we use `session.widget` to select the `CustomerId` as our hierarchy and `QuantitySold.SUM` as our measure, we can see output as expected.

In [34]:
session.widget

Open the notebook in JupyterLab with the Atoti JupyterLab extension enabled to build this widget.

However if we set the `visible` property of the `CustomerId` hierarchy to `False`, it will hide it from our list of available hierarchies as expected. 

💡 **Note:** Notice that the total number of hierarchies in the data model (`23`) does not change whether or not the `visible` property for a hierarchy is set to `True` or `False`, and that the previous `session.widget` still shows the analysis for quantity sold per customer, as the hierarchy is not **deleted** but **hidden**.

In [35]:
h["CustomerId"].visible = False

<img src="https://data.atoti.io/notebooks/hierarchies/images/hierarchies-visible-false.png" alt="hierarchies-visible-false" width="225"/>

And that's it for now! 🎉

We hope that this walk through gives you a jump start for working with hierarchies in Atoti 😊. Stay tuned for more advanced topics on Atoti hierarchies in the future! Until then, take care! 👋

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=hierarchies" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/Your-turn-to-try-Atoti.jpg" alt="Try Atoti"></a></div>